In [ ]:
## Want to try a bulk download example?
import requests
from tqdm import tqdm
import datetime

import csv
import os
from multiprocessing import Lock
from concurrent.futures import ThreadPoolExecutor
from timeit import default_timer as timer
from copy import deepcopy
from time import sleep

In [ ]:
payloadtemplate = {
    "maintext1": "Placeholder",
    "shape1": "facebook",
    "textcolor1": "black",
    "transparency1": "medium",
    "stockart1": "real-estate-ranch.jpg"
}


In [ ]:
baseurl = "http://localhost:5000/generate/"

downloaddir = "downloads/"
os.makedirs(downloaddir, exist_ok=True)

wordfile = "payload-test-bulk-parallel-words.txt"

In [ ]:
# Retrieve prepared CSV file with "filename" and "words" fields
with open(wordfile, "r") as infile:
    targetentries = list(csv.DictReader(infile))

In [ ]:
def send_text(mytext, myfilename):
    mypayload = deepcopy(payloadtemplate)
    mypayload['maintext1'] = mytext
    sleep(random.random()/50)   # Sleep a tiny bit of a second, to try to stagger our requests just a hair
    r = requests.get(baseurl, data=mypayload, timeout=15)
    while r.status_code != 200:
        sleep(random.random()/10)  # Sleep a fraction of a second
        r = requests.get(baseurl, data=mypayload, timeout=15)
    # filename = downloaddir + r.headers['Content-Disposition'].split("filename=")[-1]
    filename = myfilename
    if os.path.exists(filename):      # Do not clobber an existing file, for all sorts of reasons
        print(f"Will NOT overwrite {filename}")
    else:
        with open(filename, "wb") as outfile:
            outfile.write(r.content)
    return()

In [ ]:
print(f"Trying to publish {len(targetentries):,} items")

starttime = timer()

with ThreadPoolExecutor(max_workers=12) as executor:
    for targetentry in targetentries:        
        result = executor.submit(send_text, targetentry['words'], targetentry['filename'])
        
endtime = timer()
print(f"Time to publish {len(targetentries):,} items: {(endtime-starttime):,} seconds")